In [ ]:
%load_ext autoreload
%autoreload 2

%connect_info

In [ ]:
import django
django.setup()

In [ ]:
from typing import List

In [ ]:
projects_with_configurations = [
    {
        "project_sheet": "Finman Project Configuration",
        "configuration_sheets": [
            "FinMan Configuration_v4 Dev Loading",
#             "Vendor Configuration_v1 - Test"
        ]
    }
]

In [ ]:
"""
Populate tasks and boards related data
"""

from django.db import transaction


@transaction.atomic()
def populate_task_and_boards_related_data(configuration_sheets: List[str]):
    from ib_tasks.populate.population_utils import populate_data
    from ib_boards.populate.get_data_from_sheet_for_boards_and_columns import GetBoardsAndColumnsDataFromSheet
    
    for configuration_sheet in configuration_sheets:
        populate_data(configuration_sheet)
        
        loader = GetBoardsAndColumnsDataFromSheet()
        loader.get_data_from_sheet(configuration_sheet)
        loader.create_boards_and_columns(configuration_sheet)

In [ ]:
"""
Populate project related data
"""
from django.db import transaction

@transaction.atomic()
def populate_project_related_data(project_sheet: str):

    from ib_tasks.populate.population_utils import populate_projects_for_task_templates
    populate_projects_for_task_templates(project_sheet)
    
    from ib_boards.populate.get_data_from_sheet_for_project_boards import GetSheetDataForProjectBoards
    loader = GetSheetDataForProjectBoards()
    loader.get_data_from_project_boards_sub_sheet(project_sheet)

In [ ]:
"""
Populate Projects Related Tasks and Boards Data
"""
from django.db import transaction

@transaction.atomic()
def populate_tasks_and_borads_data_for_projects(projects_with_configurations: List[dict]):
    for project_with_configuration in projects_with_configurations:
        
        populate_task_and_boards_related_data(
            configuration_sheets=project_with_configuration["configuration_sheets"]
        )
        populate_project_related_data(
            project_sheet=project_with_configuration["project_sheet"]
        )

In [ ]:
populate_tasks_and_borads_data_for_projects(projects_with_configurations=projects_with_configurations)

In [ ]:
"""
Populate Stage Action Logic
"""

from ib_tasks.populate.write_stage_actions import writing_data_to_stage_actions_logic
writing_data_to_stage_actions_logic()